<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=882dffa08e16924a839bc3e2d711d9759f68d44cda5cd5f8ce4e62f5ede7256b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 12:16:53
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -6.76 K (-0.05%)
Current PnL: -20.21 L (-13.24%)
CY Booked + Current PnL: -6.69 L (-4.39%)
-------------------
Total profit:  1.89 L
Total loss:  -22.10 L
-------------------
Total Booked + Current PnL: 20.29 L (16.15%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.09 L (62.73%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.90,160871.0,-13688.0,13722.0,0.54,-7.84,8.53,0.02,234.0,-1.00,1.11,39.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,3.09,86912.0,-13865.0,13958.0,0.58,-13.76,16.06,0.09,253.0,-0.99,0.60,14.18,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.54,44.0,X-MC,3.46,88110.0,-2568.0,17472.0,-0.68,-2.83,19.83,16.44,63.0,-0.15,0.61,16.77,X40,ATH,PHARMA
43,ITC,452.00,-40.74,45.0,X-LC,0.91,198632.0,-1506.0,22406.0,0.12,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG
35,ICICIGI,2252.93,-16.02,54.0,X-MC,7.48,186921.0,12601.0,22599.0,-1.17,7.23,12.09,20.19,68.0,0.56,1.29,23.44,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.0,37.49,72.0,H-SC,15.14,283424.0,2301.0,130573.0,4.02,0.82,46.07,47.26,171.0,0.02,1.95,51.85,XY24,NTT,INSURANCE
83,VOLTAS,1530.0,-3.56,49.0,X-MC,3.81,204990.0,13248.0,24517.0,2.22,6.91,11.96,19.69,78.0,0.54,1.41,14.18,XY25,NTT,AC
84,WHIRLPOOL,2270.0,-41.57,40.0,M-SC,9.30,114476.0,-322.0,94363.0,1.39,-0.28,82.43,81.92,236.0,-0.00,0.79,35.52,XR,NTT,DURABLES
45,JIOFIN,387.0,-0.59,60.0,X-LC,9.17,270384.0,3019.0,62432.0,1.19,1.13,23.09,24.48,36.0,0.05,1.86,56.72,XY24,BTT,FINANCE
5,ANGELONE,3033.0,59.94,74.0,X-SC,7.21,224614.0,33608.0,24101.0,0.95,17.60,10.73,30.21,97.0,1.39,1.55,42.15,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,2275.00,-20.32,53.0,X-LC,4.21,320054.0,7166.0,164508.0,-2.54,2.29,51.40,54.87,2.0,0.04,2.20,10.76,X40,BTT,IT
64,SFL,1287.00,27.14,39.0,M-SC,3.04,167958.0,-94260.0,164095.0,-2.38,-35.95,97.70,26.63,239.0,-0.57,1.16,4.43,XY24,NTT,MISC
77,TRIDENT,48.00,-10.90,45.0,M-SC,1.90,69432.0,-22822.0,49130.0,-2.29,-24.74,70.76,28.51,225.0,-0.46,0.48,18.81,XR,NTT,TEXTILES
61,ROUTE,2227.21,-57.12,39.0,H-SC,23.06,70367.0,-62699.0,154582.0,-2.22,-47.12,219.68,69.05,140.0,-0.41,0.48,3.68,SR,ATH,IT
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.35,85473.0,-43197.0,88490.0,-1.59,-33.57,103.53,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114476.0,-322.0,94363.0,1.39,-0.28,82.43,81.92,236.0,-0.00,0.79,35.52,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.33,147450.0,-194.0,159526.0,-0.10,-0.13,108.19,107.91,216.0,-0.00,1.02,2.86,OX40N,ATH,CABLES
69,STARHEALTH,761.00,37.49,72.0,H-SC,15.14,283424.0,2301.0,130573.0,4.02,0.82,46.07,47.26,171.0,0.02,1.95,51.85,XY24,NTT,INSURANCE
86,ZYDUSLIFE,1286.17,-14.08,38.0,H-MC,5.75,209199.0,-375.0,75040.0,0.26,-0.18,35.87,35.63,119.0,-0.00,1.44,16.46,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,51.0,M-LC,5.65,151013.0,68.0,109394.0,-0.72,0.04,72.44,72.51,99.0,0.00,1.04,6.07,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.33,147450.0,-194.0,159526.0,-0.10,-0.13,108.19,107.91,216.0,-0.00,1.02,2.86,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.90,160871.0,-13688.0,13722.0,0.54,-7.84,8.53,0.02,234.0,-1.00,1.11,39.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,3.09,86912.0,-13865.0,13958.0,0.58,-13.76,16.06,0.09,253.0,-0.99,0.60,14.18,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.79,52.0,H-SC,2.48,224217.0,-45450.0,81772.0,-0.08,-16.85,36.47,13.47,158.0,-0.56,1.54,14.01,XY24,NTT,PAINTS
67,SIS,528.00,2016.58,46.0,H-SC,2.43,85425.0,-25607.0,49213.0,-0.27,-23.06,57.61,21.26,166.0,-0.52,0.59,15.42,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.49,72.0,H-SC,15.14,283424.0,2301.0,130573.0,4.02,0.82,46.07,47.26,171.0,0.02,1.95,51.85,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,18.70,55.0,M-SC,36.51,159885.0,10665.0,133520.0,-0.38,7.15,83.51,96.62,245.0,0.08,1.10,73.03,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.25,51.0,M-LC,5.65,151013.0,68.0,109394.0,-0.72,0.04,72.44,72.51,99.0,0.00,1.04,6.07,XR,NTT,IT
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.61,130433.0,7097.0,124524.0,-0.65,5.75,95.47,106.72,139.0,0.06,0.90,30.48,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.61,130433.0,7097.0,124524.0,-0.65,5.75,95.47,106.72,139.0,0.06,0.90,30.48,AR,ATH,MISC
25,FINCABLES,1641.55,-18.93,47.0,M-SC,10.33,147450.0,-194.0,159526.0,-0.10,-0.13,108.19,107.91,216.0,-0.00,1.02,2.86,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114476.0,-322.0,94363.0,1.39,-0.28,82.43,81.92,236.0,-0.00,0.79,35.52,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,51.0,M-LC,5.65,151013.0,68.0,109394.0,-0.72,0.04,72.44,72.51,99.0,0.00,1.04,6.07,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.08,38.0,H-MC,5.75,209199.0,-375.0,75040.0,0.26,-0.18,35.87,35.63,119.0,-0.00,1.44,16.46,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.35,85473.0,-43197.0,88490.0,-1.59,-33.57,103.53,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,29.0,X-SC,37.28,46691.0,-18315.0,169246.0,-0.76,-28.17,362.48,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
26,GILLETTE,11206.78,-17.51,30.0,X-SC,5.89,254940.0,1044.0,81275.0,0.41,0.41,31.88,32.42,85.0,0.01,1.76,15.57,X40,ATH,FMCG
27,GLAXO,3466.20,-19.47,30.0,X-MC,5.82,200000.0,716.0,77300.0,0.22,0.36,38.65,39.15,60.0,0.01,1.38,29.47,X40,ATH,PHARMA
23,DMART,5391.45,-17.36,31.0,X-LC,5.63,201560.0,-8847.0,68006.0,-0.77,-4.20,33.74,28.12,19.0,-0.13,1.39,18.43,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-15.75,40.0,X-LC,0.11,258201.0,-18209.0,439381.0,-0.94,-6.59,170.17,152.37,3.0,-0.04,1.78,0.86,XY24,NTT,AUTO
43,ITC,452.00,-40.74,45.0,X-LC,0.91,198632.0,-1506.0,22406.0,0.12,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG
34,HONAUT,58357.33,-24.86,37.0,X-SC,0.95,106740.0,-21198.0,68335.0,0.18,-16.57,64.02,36.84,90.0,-0.31,0.74,9.28,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.34,66.0,X-MC,1.00,184194.0,7456.0,47283.0,-0.23,4.22,25.67,30.97,75.0,0.16,1.27,17.31,X40,ATH,INSURANCE
16,CAMS,4762.00,-1.25,58.0,X-SC,1.62,261096.0,9240.0,53185.0,0.30,3.67,20.37,24.79,86.0,0.17,1.80,27.66,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.35,85473.0,-43197.0,88490.0,-1.59,-33.57,103.53,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,29.0,X-SC,37.28,46691.0,-18315.0,169246.0,-0.76,-28.17,362.48,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
53,PAGEIND,51769.92,-28.58,37.0,X-MC,7.68,158760.0,-4980.0,48327.0,0.27,-3.04,30.44,26.47,55.0,-0.10,1.09,0.59,X40,ATH,APPARELS
76,TMPV,1065.00,-15.75,40.0,X-LC,0.11,258201.0,-18209.0,439381.0,-0.94,-6.59,170.17,152.37,3.0,-0.04,1.78,0.86,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-18.37,36.0,X-MC,6.54,176300.0,-24206.0,121365.0,-0.13,-12.07,68.84,48.45,64.0,-0.20,1.21,1.66,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,29.0,X-SC,37.28,46691.0,-18315.0,169246.0,-0.76,-28.17,362.48,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.54,38.0,X-SC,3.45,79712.0,-65448.0,141377.0,-1.29,-45.09,177.36,52.31,91.0,-0.46,0.55,6.70,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,12.06,27.0,X-SC,9.35,85473.0,-43197.0,88490.0,-1.59,-33.57,103.53,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.54,44.0,X-MC,3.46,88110.0,-2568.0,17472.0,-0.68,-2.83,19.83,16.44,63.0,-0.15,0.61,16.77,X40,ATH,PHARMA
34,HONAUT,58357.33,-24.86,37.0,X-SC,0.95,106740.0,-21198.0,68335.0,0.18,-16.57,64.02,36.84,90.0,-0.31,0.74,9.28,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,59.0,X-LC,8.64,291419.0,-54537.0,121230.0,-0.18,-15.76,41.60,19.28,1.0,-0.45,2.01,7.73,X40,ATH,IT
41,INFY,2275.00,-20.32,53.0,X-LC,4.21,320054.0,7166.0,164508.0,-2.54,2.29,51.40,54.87,2.0,0.04,2.20,10.76,X40,BTT,IT
76,TMPV,1065.00,-15.75,40.0,X-LC,0.11,258201.0,-18209.0,439381.0,-0.94,-6.59,170.17,152.37,3.0,-0.04,1.78,0.86,XY24,NTT,AUTO
82,VBL,671.64,-18.58,46.0,X-LC,6.07,291662.0,-24180.0,136848.0,0.92,-7.66,46.92,35.67,4.0,-0.18,2.01,5.14,X40N,ATH,FMCG
43,ITC,452.00,-40.74,45.0,X-LC,0.91,198632.0,-1506.0,22406.0,0.12,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-704.67,75.0,L-MC,6.49,48977.0,-28230.0,53625.0,-0.49,-36.56,109.49,32.89,259.0,-0.53,0.34,34.88,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,44.0,L-SC,21.80,77280.0,-36269.0,76322.0,-0.45,-31.94,98.76,35.27,270.0,-0.48,0.53,89.14,XR,NTT,HOTELS
7,ASIANTILES,137.00,7516.67,62.0,L-SC,16.40,83597.0,-10213.0,86832.0,0.19,-10.89,103.87,81.67,271.0,-0.12,0.58,61.89,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.70,51.0,H-SC,7.48,92520.0,-5460.0,27062.0,0.70,-5.57,29.25,22.05,164.0,-0.20,0.64,34.48,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.58,53.0,H-SC,4.61,130433.0,7097.0,124524.0,-0.65,5.75,95.47,106.72,139.0,0.06,0.90,30.48,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.49,86.0,X-LC,8.83,255490.0,3722.0,49488.0,0.83,1.48,19.37,21.13,31.0,0.08,1.76,37.43,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,145.54,84.0,M-SC,1.90,160871.0,-13688.0,13722.0,0.54,-7.84,8.53,0.02,234.0,-1.00,1.11,39.93,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,71.90,60.0,H-SC,8.46,149206.0,-33569.0,144640.0,-0.28,-18.37,96.94,60.77,159.0,-0.23,1.03,34.37,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7516.67,62.0,L-SC,16.40,83597.0,-10213.0,86832.0,0.19,-10.89,103.87,81.67,271.0,-0.12,0.58,61.89,XR,NTT,CERAMICS
40,INDUSINDBK,1800.00,-704.67,75.0,L-MC,6.49,48977.0,-28230.0,53625.0,-0.49,-36.56,109.49,32.89,259.0,-0.53,0.34,34.88,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.58
1,25,44.49
2,50,75.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.61
MC    27.70
LC    24.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.48
X40      20.51
X40N     12.24
XY25     10.05
XR        9.57
AR        8.98
OX40N     7.55
X200      1.80
SR        1.00
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.23
X-MC    21.35
X-LC    19.81
M-SC    13.35
X-SC     8.58
H-MC     4.65
H-LC     2.86
L-SC     1.45
M-MC     1.36
M-LC     1.04
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.61,-4.39,38.15
IT,12.46,-16.03,78.06
FINANCE,11.94,-9.11,58.57
MISC,7.54,-18.75,78.65
PAINTS,6.02,-5.61,21.35
ELECTRICAL,5.55,-12.10,51.84
INSURANCE,4.51,3.42,30.63
AUTO,3.46,-14.30,123.88
PHARMA,3.43,-0.45,34.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3314519.0,20
XR,1268036.0,13
AR,1266779.0,10
X40,992532.0,14
OX40N,698550.0,10
X40N,693078.0,9
XY25,474169.0,7
SR,274025.0,2
MH,65242.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3486214.0,25
M-SC,1413744.0,16
X-LC,1191898.0,11
X-MC,1142342.0,15
X-SC,831718.0,8
H-MC,402749.0,3
L-SC,251152.0,3
H-LC,141479.0,2
M-LC,109394.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167142.0      6
           AR         842866.0      5
           XR         798370.0      7
M-SC       XY24       739085.0      6
X-LC       XY24       501813.0      2
X-MC       X40        413241.0      7
X-LC       X40        409526.0      5
X-SC       XY24       374955.0      2
X-MC       XY24       347617.0      3
H-SC       OX40N      338569.0      4
X-SC       X40N       286998.0      4
H-SC       SR         274025.0      2
M-SC       OX40N      271983.0      5
H-MC       AR         218842.0      2
X-LC       X40N       204854.0      2
X-MC       X40N       201226.0      3
H-MC       XY24       183907.0      1
X-MC       XY25       180258.0      2
X-SC       X40        169765.0      2
L-SC       XR         163154.0      2
M-SC       XR         143493.0      2
           XY25       133520.0      1
           AR         125663.0      2
M-LC       XR         109394.0      1
L-SC       OX40N       87998.0      1
H-LC       AR          79408.0      1
X-LC       XY25        75705.0      2
H-SC       MH          65242.0      1
H-LC       X200        62071.0      1
L-MC       XR          53625.0      1
M-MC       XY25        47829.0      1
L-LC       XY25        36857.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
